In [1]:
import pandas as pd

# Load the dataset
file_path = 'exchange_rate.csv'
df = pd.read_csv(file_path, parse_dates=[0])

# Display the first few rows to understand the structure
df.head()

,date,Ex_rate
0,01-01-1990 00:00,0.7855
1,02-01-1990 00:00,0.7818
2,03-01-1990 00:00,0.7867
3,04-01-1990 00:00,0.7860
4,05-01-1990 00:00,0.7849


In [ ]:
## Data Preparation and Exploration

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Seting the date as index
df.set_index('date', inplace=True)

In [ ]:
# Checking for missing values
missing_values = df.isnull().sum()

In [ ]:
# Plot the time series
plt.figure(figsize=(14, 6))
plt.plot(df.index, df['Ex_rate'], label='USD to AUD Exchange Rate')
plt.title('USD to AUD Exchange Rate Over Time')
plt.xlabel('Date')
plt.ylabel('Exchange Rate')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

missing_values


In [ ]:
## Model Building - ARIMA

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller

# Checking for stationarity with Augmented Dickey-Fuller test
adf_result = adfuller(df['Ex_rate'])
adf_result_summary = {
    'ADF Statistic': adf_result[0],
    'p-value': adf_result[1],
    'Critical Values': adf_result[4]
}

# If p-value > 0.05, difference the series
df_diff = df['Ex_rate'].diff().dropna()

# Plot ACF and PACF of differenced series
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
plot_acf(df_diff, ax=axes[0], lags=40)
plot_pacf(df_diff, ax=axes[1], lags=40)
axes[0].set_title('ACF - Differenced Series')
axes[1].set_title('PACF - Differenced Series')
plt.tight_layout()
plt.show()

adf_result_summary


In [ ]:
from statsmodels.tsa.arima.model import ARIMA
import numpy as np

In [ ]:
# Fit ARIMA(1,1,1)
model_arima = ARIMA(df['Ex_rate'], order=(1, 1, 1))
arima_result = model_arima.fit()

In [ ]:
# Diagnostics: plot residuals
residuals = arima_result.resid

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
sns.histplot(residuals, bins=30, kde=True, ax=axes[0])
axes[0].set_title('Histogram of ARIMA Residuals')
axes[0].set_xlabel('Residuals')

sns.lineplot(x=np.arange(len(residuals)), y=residuals, ax=axes[1])
axes[1].set_title('Residuals Over Time')
axes[1].set_xlabel('Time')
axes[1].set_ylabel('Residuals')
plt.tight_layout()
plt.show()

In [ ]:
# Forecasting the next 12 months
forecast_arima = arima_result.get_forecast(steps=12)
forecast_index = pd.date_range(df.index[-1], periods=12, freq='MS')
forecast_mean = forecast_arima.predicted_mean
forecast_ci = forecast_arima.conf_int()


In [ ]:
# Plot forecast
plt.figure(figsize=(14, 6))
plt.plot(df.index, df['Ex_rate'], label='Observed')
plt.plot(forecast_index, forecast_mean, label='Forecast (ARIMA)', color='orange')
plt.fill_between(forecast_index, forecast_ci.iloc[:, 0], forecast_ci.iloc[:, 1], color='orange', alpha=0.3)
plt.title('ARIMA Forecast of USD to AUD Exchange Rate')
plt.xlabel('Date')
plt.ylabel('Exchange Rate')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()